# Exercice API

In [37]:
import pandas as pd
import requests
from math import ceil

In [14]:
url = "https://gist.github.com/paulmillr/2657075"
dfs = pd.read_html(url, attrs={"cellspacing":"0"})
assert len(dfs) == 1, "there are multiple results from read_html"

In [15]:
most_active_users = dfs[0]
most_active_users.head()

,#,User,Contribs,Location,Picture
0,#1,fabpot (Fabien Potencier),12947,San Francisco,NaN
1,#2,andrew (Andrew Nesbitt),10188,"Somerset, UK",NaN
2,#3,taylorotwell (Taylor Otwell),8376,"Little Rock, AR",NaN
3,#4,egoist (EGOIST),7688,"Chengdu, China",NaN
4,#5,HugoGiraudel (Hugo Giraudel),7658,Berlin,NaN


In [16]:
testUsers = ["fabpot", "radcheb"]

In [89]:
!curl -s "https://api.github.com/users/fabpot?per_page=100" | jq ".public_repos" && curl -s "https://api.github.com/users/fabpot/repos?per_page=100" | jq ".[].stargazers_count" | awk '{s+=$1} END {print s}'

36
923


In [90]:
!curl -s "https://api.github.com/users/radcheb?per_page=100" | jq ".public_repos" && curl -s "https://api.github.com/users/radcheb/repos?per_page=100" | jq ".[].stargazers_count" | awk '{s+=$1} END {print s}'

82
3


In [100]:
expected_mean_stars = [923/36, 3/82]

In [25]:
import getpass
github_token = getpass.getpass()
headers = {'Authorization': 'token ' + github_token}

········


In [109]:
def get_user_repos_count(username):
    url = "https://api.github.com/users/" + username
    response = requests.get(url, headers=headers)
    return response.json()['public_repos']
def get_mean_stars(username):
    num_pages = ceil(get_user_repos_count(username) / 100)
    url_template =  "https://api.github.com/users/{0}/repos?per_page=100&page={1}"
    repos_stars = []
    for page in range(0, num_pages):
        url = url_template.format(username, page)
        response = requests.get(url, headers=headers)
        repos_stars.extend([repo['stargazers_count'] for repo in response.json()])
    return sum(repos_stars) / len(repos_stars) if len(repos_stars) > 0 else 0  

In [103]:
actual_mean_stars = [get_mean_stars(u) for u in testUsers]
assert actual_mean_stars == expected_mean_stars, "resulat is not same as expected"

In [110]:
most_active_users['mean_stars'] = most_active_users['User'].apply(lambda u: get_mean_stars(u.split(" ")[0]))
most_active_users.head()

,#,User,Contribs,Location,Picture,mean_stars
0,#1,fabpot (Fabien Potencier),12947,San Francisco,NaN,25.638889
1,#2,andrew (Andrew Nesbitt),10188,"Somerset, UK",NaN,3.427500
2,#3,taylorotwell (Taylor Otwell),8376,"Little Rock, AR",NaN,4.500000
3,#4,egoist (EGOIST),7688,"Chengdu, China",NaN,31.215714
4,#5,HugoGiraudel (Hugo Giraudel),7658,Berlin,NaN,152.725000


In [73]:
url = "https://api.github.com/users/radcheb/repos?per_page=100&page=1"
response = requests.get(url, headers=headers)

In [113]:
most_active_users[['User', 'mean_stars']].sort_values('mean_stars', ascending=False)

,User,mean_stars
225,bailicangdu (cangdu),3502.923077
18,kamranahmedse (Kamran Ahmed),2274.250000
142,vinta (Vinta),1829.906250
117,mrdoob (Mr.doob),1597.057143
194,IanLunn (Ian Lunn),1401.944444
44,sebastianbergmann (Sebastian Bergmann),1387.633333
230,junyanz (Jun-Yan Zhu),1372.857143
153,nolimits4web (Vladimir Kharlampidi),1338.642857
195,ruanyf (Ruan YiFeng),1260.830189
109,JakeWharton (Jake Wharton),1072.860000
